![Banner](docs/Banner.png)

# Dolphin: InSAR phase linking library for creating surface displacement maps using persistent scatterer (PS) and distributed scatterer (DS) processing

**Author:** Scott Staniewicz, Sara Mirzaee, August 18-22, 2025 [EarthScope InSAR Short Course (ISCE+)](https://www.earthscope.org/event/insar-processing-and-analysis-isce/).

dolphin is an open-source package for InSAR time-series analysis and is the core software used to produce OPERA displacement products. The inputs are stacks of coregistered SLCs (in either geo- or radar-coordinates) in all gdal compatible formats and outputs are displacement time-series without corrections. 

dolphin is available on Github from the following page: https://github.com/isce-framework/dolphin

References: 

+ S. Staniewicz, S. Mirzaee, G. M. Gunter, T. O. Cabrera, E. Havazli, H.Fattahi ., (2024). Dolphin :
A Python package for large-scale InSAR PS/DS processing. Journal of Open Source Software,
9(103), 6997. [ [doi](https://doi.org/10.21105/joss.06997) ]

# Table of Contents

+ [0. Initial setup](#0.-Initial-setup-of-the-notebook)
+ [1. General overview](#1.-General-overview-of-dolphin)
   - [1.1 Processing steps](#1.1-Processing-steps-of-dolphin)
+ [2. Time series analysis](#2.-Time-series-analysis---Phase-Linking)
+ [3. dolphin subcommnds](#3.-Other-dolphin-commands)


# 0. Initial setup of the notebook

The cell below performs the intial setup of the notebook and must be **run every time the notebook (re)starts**. It defines the processing location and check the example dataset.

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal
from itertools import islice
from pathlib import Path

plt.rcParams.update({'font.size': 12})

# define and go to the work directory
work_dir = Path("./").expanduser()
work_dir.mkdir(parents=True, exist_ok=True)

# define the custom config file
src_cfg = Path("dolphin_config.yaml")
config_file = work_dir / src_cfg.name
print("Work directory:", work_dir)


# --- GDAL viewer (accepts str or Path) ---
def view_gdal(file, v_range=None, ref=None, mask=None, bbox=None):
    """
    file: str | Path - raster to view
    v_range: (vmin, vmax) or None
    ref: (row, col) or None - subtract value at this pixel
    mask: str | Path or None - mask raster (nonzero/1 keeps data)
    bbox: (xmin, ymin, xmax, ymax) in pixel indices or None - crop [x0:x1, y0:y1]
    """
    file = Path(file)
    ds = gdal.Open(str(file), gdal.GA_ReadOnly)
    if ds is None:
        raise FileNotFoundError(f"Cannot open raster: {file}")

    data = ds.GetRasterBand(1).ReadAsArray()

    if ref is not None:
        data = data.astype(float, copy=False)
        data -= data[ref[0], ref[1]]

    if mask is not None:
        mask = Path(mask)
        dsmask = gdal.Open(str(mask), gdal.GA_ReadOnly)
        if dsmask is None:
            raise FileNotFoundError(f"Cannot open mask: {mask}")
        mask_array = dsmask.GetRasterBand(1).ReadAsArray()
        data = np.ma.masked_array(data, 1 - mask_array)

    if bbox is not None:
        # bbox = (x0, y0, x1, y1)
        data = data[bbox[1]:bbox[3], bbox[0]:bbox[2]]

    if v_range is None:
        plt.imshow(data, interpolation="none", cmap="RdBu_r")
    else:
        plt.imshow(data, vmin=v_range[0], vmax=v_range[1], interpolation="none", cmap="RdBu_r")
    plt.colorbar()
    plt.show()

Work directory: .


In [12]:
# download the staged data from AWS 
tar_file = work_dir / 'NewOrleans.tar.gz'
if tar_file.exists():
    print('Staged RSLC data exists at: {}'.format(tar_file))
else:
    !aws s3 cp s3://earthscope-insar2025/dolphin-isce2/NewOrleans.tar.gz ./


# decompress the tar file [it takes ~1.5 min]
print('decompressing the downloaded dataset...')
!pv NewOrleans.tar.gz | tar -xz


Staged RSLC data exists at: NewOrleans.tar.gz
decompressing the downloaded dataset...
23.6GiB 0:06:44 [59.9MiB/s] [================================>] 100%            


# 1. General overview of dolphin

This application provides a workflow for phase linking which includes two main steps: creating a config file and running the workflow with the configuration parameters set. 

Configuration parameters for each step are initiated with default values in a customizable text file (dolphin_config.yaml). In this notebook, we will walk through the various steps.


![flowchart](docs/flowchart.png)

<p style="text-align: center;">
    (Figure from Scott J. Staniewicz et al., 2024)
</p>

## 1.1 Processing steps of dolphin

Running all subcommands with "--help", shows the full command line. 

In [ ]:
!dolphin --help

The main subcommands to run the workflow are `config` and `run`. An empty configuration yaml file can be generated with `--print-empty` or by specifying the input coregistered SLCs at minimum.

In [10]:
# !dolphin config --print-empty
# !dolphin config --cslc ./data/SLC/*/*.tif
yaml_path = Path("dolphin_config.yaml")
for line in islice(yaml_path.open(), 60):  # change 40 as needed
    print(line.rstrip())

input_options:
  # If passing HDF5/NetCDF files, subdataset to use from CSLC files. .
  #   Type: string | null.
  subdataset:
  # Format of dates contained in CSLC filenames.
  #   Type: string.
  cslc_date_fmt: '%Y%m%d'
  # Radar wavelength (in meters) of the transmitted data. used to convert the units in the
  #   rasters in `timeseries/` to from radians to meters. If None and sensor is not
  #   recognized, outputs remain in radians.
  #   Type: number | null.
  wavelength: 0.056
# Mask file used to ignore low correlation/bad data (e.g water mask). Convention is 0 for no
#   data/invalid, and 1 for good data. Dtype must be uint8.
#   Type: string | null.
mask_file:
# Name of sub-directory to use for writing output files.
#   Type: string.
work_directory: ./
worker_settings:
  # Whether to use GPU for processing (if available).
  #   Type: boolean.
  gpu_enabled: true
  # Number of threads to use per worker. This sets the OMP_NUM_THREADS environment variable in
  #   each python pro



From the config parameters, following are the main parameters you may want to set before running the workflow. 
1. cslc_file_list
2. wavelength  (if you don't set it, output timeseries are in radians unit)
3. strides (to generate a lower resolution product by selecting PS and DS pixels wisely in a moving window)
4. half_window (for finding statistically homogeneous pixels)
5. interferogram_network (generate a network based on your application requirements for unwrapping. set reference_idx for a single reference network, max_bandwidth for a nearest-n network)

# 2. Time series analysis - Phase Linking

Run the whole workflow from reading input coregistered SLCs to output displacement timeseries and velocity map.
There is another folder in the input data which refers to corresponding geometry files generated by ISCE2. They are not generally required 

In [13]:
!dolphin run dolphin_config.yaml

[INFO|displacement|L117] 2025-08-19T21:05:36+0000: Found SLC files from 1 bursts
[INFO|wrapped_phase|L103] 2025-08-19T21:05:36+0000: Running wrapped phase estimation in /home/jovyan/Sara_work/2025-isceplus/5.3_Intro_to_Dolphin/isce2-dolphin/phase_linking
[WARNING|wrapped_phase|L553] 2025-08-19T21:05:36+0000: Could not make nodata mask: /home/jovyan/Sara_work/2025-isceplus/5.3_Intro_to_Dolphin/isce2-dolphin/data/SLC/20241223/20241223.slc.tif is not a CSLC file
[INFO|wrapped_phase|L149] 2025-08-19T21:05:36+0000: Creating persistent scatterer file /home/jovyan/Sara_work/2025-isceplus/5.3_Intro_to_Dolphin/isce2-dolphin/phase_linking/PS/ps_pixels.tif
[INFO|_readers|L1090] 2025-08-19T21:05:36+0000: Processing (512, 512) sized blocks...
PS (/home/jovyan/Sara_work/2025-isceplus/5.3_Intro_to_Dolphin/isce2-dolphin/phas
[INFO|ps|L159] 2025-08-19T21:11:34+0000: Waiting to write 0 blocks of data.
[INFO|ps|L162] 2025-08-19T21:11:34+0000: Repacking PS rasters for better compression
/opt/conda/envs/ea

The outputs of the workflow are displacement timeseries and velocity map. There are byproducts includig the inversion residuals, a conected component intersection from all unwrapped interferograms, PS mask, temporal coherence, shp counts, phase similarity for all ministacks and crlb. You can also find the amplitude dispersion used for finding PS pixels.

#### Velocity map
![Velocity_map](docs/velocity_map.png)

# 3. Other dolphin commands
1. You can run snaphu on all interferograms in a directory and unwrap them with the `unwrap` subcommand.
2. You can invert the unwrapped interferograms, estimate timeseries and phase velocity with the `timeseries` subcommand
3. You can filter a list of unwrapped interferogram files using a long-wavelength filter with the `filter` subcommnd and generate short wavelength products

In [2]:
# dolphin unwrap --help
# dolphin timeseries --help
# dolphin filter --help